In [5]:
# settings.py
import os
import pandas as pd
from os.path import join, dirname
from dotenv import load_dotenv

dotenv_path = join(dirname(__file__), '../config/dev.env')
load_dotenv(dotenv_path)

import warnings
warnings.filterwarnings('ignore')

## query your data without llm (from Deeplearning AI Q&A)


In [6]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [9]:
file = './data/dummy_data-Copy1.csv'
loader = CSVLoader(file_path=file)

In [10]:
df = pd.read_csv(file)
print(df.shape)
df.head()

(20, 6)


,Name,Age,Gender,Income,ozip,dzip
0,Alice,25,F,50000.0,55345.0,56002.0
1,Bob,-5,M,55000.0,56001.0,55345.0
2,Carol,30,F,NaN,NaN,NaN
3,David,128,M,60000.0,NaN,NaN
4,Eve,29,F,65000.0,NaN,NaN


In [106]:
from langchain.indexes import VectorstoreIndexCreator

In [107]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [108]:
query ="Please list all Men who make over $100000\
in a table in markdown and summarize each one."

In [109]:
response = index.query(query)
response

'\n\n| Name | Age | Gender | Income |\n|------|-----|--------|--------|\n| Nick | 66  | M      | 100000 |\n\nNick is a 66 year old man with an income of $100000.\n\n| Name | Age | Gender | Income |\n|------|-----|--------|--------|\n| David | 128  | M      | 60000 |\n\nDavid is a 128 year old man with an income of $60000.'

In [110]:
display(Markdown(response))



| Name | Age | Gender | Income |
|------|-----|--------|--------|
| Nick | 66  | M      | 100000 |

Nick is a 66 year old man with an income of $100000.

| Name | Age | Gender | Income |
|------|-----|--------|--------|
| David | 128  | M      | 60000 |

David is a 128 year old man with an income of $60000.

In [60]:
loader = CSVLoader(file_path=file)

In [88]:
docs = loader.load()

In [111]:
docs[0]

Document(page_content='Name: Alice\nAge: 25\nGender: F\nIncome: 50000', metadata={'source': './data/dummy_data.csv', 'row': 0})

In [129]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [130]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [131]:
print(len(embed))
print(embed[:5])

1536
[-0.021913960576057434, 0.006774206645786762, -0.018190348520874977, -0.039148248732089996, -0.014089343138039112]


In [132]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [133]:
query = "Please suggest someone similar to Josh he is 40 years old and makes $100000"

In [134]:
docs = db.similarity_search(query)
docs[0]

Document(page_content='Name: Sam\nAge: 60\nGender: M\nIncome: 120000', metadata={'source': './data/dummy_data.csv', 'row': 18})

## user python agent

In [11]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

In [12]:
llm = ChatOpenAI(temperature = 0.0)

In [13]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [ ]:


#response = llm.call_as_llm(f"{df} Question: Please add a new column city and state for anything that looks like a zip code.  the column headings should have no spaces") 
#display(Markdown(response))

agent.run(f"{df} Create a pandas dataframe and add a new column city and state for anything that looks like a zip code. if zip value is blank, skip it.  print out the dataframe") 

In [20]:
agent.run(f"{df} use python pandas library to generate python code to get distinct age, print out the python code") 



> Entering new  chain...
To get the distinct ages from the given dataset, we can use the pandas library in Python. We can read the dataset into a pandas DataFrame and then use the `unique()` function on the 'Age' column to get the distinct values. Finally, we can print out the Python code to achieve this.

Action: Python_REPL
Action Input: 

```python
import pandas as pd

data = {
    'Name': ['Alice', 'Bob', 'Carol', 'David', 'Eve', 'Frank', 'Gina', 'Hank', 'Ivy', 'Jack', 'Kathy', 'Larry', 'Mona', 'Nick', 'Oscar', 'Pam', 'Quincy', 'Rita', 'Sam', 'Tina'],
    'Age': [25, -5, 30, 128, 29, 20, 42, 33, 37, 120, 44, 15, -10, 66, 31, 40, 25, 50, 60, 35],
    'Gender': ['F', 'M', 'F', 'M', 'F', 'M', 'F', 'M', 'F', 'M', 'F', 'M', 'F', 'M', 'M', 'F', 'M', 'F', 'M', 'F'],
    'Income': [50000.0, 55000.0, float('nan'), 60000.0, 65000.0, float('nan'), 71000.0, -10000.0, float('nan'), 80000.0, 85000.0, 90000.0, 95000.0, 100000.0, 10000.0, -20000.0, 45000.0, float('nan'), 120000.0, 40000.0],
    

'[25, -5, 30, 128, 29, 20, 42, 33, 37, 120, 44, 15, -10, 66, 31, 40, 50, 60, 35]'

## query your data with llm included

In [118]:
retriever = db.as_retriever()

In [119]:
llm = ChatOpenAI(temperature = 0.0)


In [120]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [145]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all for each in a table in markdown and pandas dataframe.  the column headings should have no spaces") 


In [146]:
display(Markdown(response))

Markdown Table:

|Name|Age|Gender|Income|
|----|---|------|------|
|Sam|60|M|120000|
|Jack|120|M|80000|
|Nick|66|M|100000|
|Oscar|31|M|10000|

Pandas DataFrame:

```python
import pandas as pd

data = {'Name': ['Sam', 'Jack', 'Nick', 'Oscar'],
        'Age': [60, 120, 66, 31],
        'Gender': ['M', 'M', 'M', 'M'],
        'Income': [120000, 80000, 100000, 10000]}

df = pd.DataFrame(data)
print(df)
```

Output:

```
   Name  Age Gender  Income
0   Sam   60      M  120000
1  Jack  120      M   80000
2  Nick   66      M  100000
3  Oscar  31      M   10000
```

In [148]:
import pandas as pd

data = {'Name': ['Sam', 'Jack', 'Nick', 'Oscar'],
        'Age': [60, 120, 66, 31],
        'Gender': ['M', 'M', 'M', 'M'],
        'Income': [120000, 80000, 100000, 10000]}

df = pd.DataFrame(data)

print(df)

    Name  Age Gender  Income
0    Sam   60      M  120000
1   Jack  120      M   80000
2   Nick   66      M  100000
3  Oscar   31      M   10000


In [92]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [93]:
query =  "Please list all Men who make over $100000\
in a table in markdown and summarize each one."

In [94]:
response = qa_stuff.run(query)



> Entering new  chain...

> Finished chain.


In [46]:
display(Markdown(response))

| Name  | Age | Gender | Income |
|-------|-----|--------|--------|
| Nick  | 66  | M      | 100000 |
| David | 128 | M      | 60000  |

- Nick is a 66-year-old man with an income of $100,000. 
- David is a 128-year-old man with an income of $60,000.

In [47]:
response = index.query(query, llm=llm)

In [48]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])
index

VectorStoreIndexWrapper(vectorstore=<langchain.vectorstores.docarray.in_memory.DocArrayInMemorySearch object at 0x7f70fc1ce9b0>)

In [45]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [3]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [37]:
import pandas as pd
filepath = './data/dummy_data.csv'
dummy_data = pd.read_csv(filepath)
dummy_data.head(2)

,Name,Age,Gender,Income
0,Alice,25,F,50000.0
1,Bob,-5,M,55000.0


Q&A from CSV

In [42]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import FAISS

loader = CSVLoader(file_path=filepath)
data = loader.load()

In [43]:
from langchain.agents import create_csv_agent
from langchain.llms import OpenAI

agent = create_csv_agent(OpenAI(temperature=0), [filepath], verbose=True)

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(data, embeddings)

ImportError: Missing optional dependency 'tabulate'.  Use pip or conda to install tabulate.

In [40]:
from langchain.indexes import VectorstoreIndexCreator
index = VectorstoreIndexCreator().from_loaders([loader])

ValueError: Could not import chromadb python package. Please install it with `pip install chromadb`.

In [27]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

In [39]:
print(data)

[Document(page_content='Name: Alice\nAge: 25\nGender: F\nIncome: 50000', metadata={'source': './data/dummy_data.csv', 'row': 0}), Document(page_content='Name: Bob\nAge: -5\nGender: M\nIncome: 55000', metadata={'source': './data/dummy_data.csv', 'row': 1}), Document(page_content='Name: Carol\nAge: 30\nGender: F\nIncome: ', metadata={'source': './data/dummy_data.csv', 'row': 2}), Document(page_content='Name: David\nAge: 128\nGender: M\nIncome: 60000', metadata={'source': './data/dummy_data.csv', 'row': 3}), Document(page_content='Name: Eve\nAge: 29\nGender: F\nIncome: 65000', metadata={'source': './data/dummy_data.csv', 'row': 4}), Document(page_content='Name: Frank\nAge: 20\nGender: M\nIncome: ', metadata={'source': './data/dummy_data.csv', 'row': 5}), Document(page_content='Name: Gina\nAge: 42\nGender: F\nIncome: 71000', metadata={'source': './data/dummy_data.csv', 'row': 6}), Document(page_content='Name: Hank\nAge: 33\nGender: M\nIncome: -10000', metadata={'source': './data/dummy_data

In [28]:
llm = ChatOpenAI(temperature=0)

In [29]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [30]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [33]:
agent.run(f"""give me python script to clean this data: {dummy_data}""")



> Entering new  chain...
To clean the data, I need to remove any invalid or missing values and ensure that the data is in the correct format.
Action: Python_REPL
Action Input: import pandas as pd
Observation: 
Thought:I have imported the pandas library, which will allow me to work with the data in a tabular format.
Action: Python_REPL
Action Input: data = {'Name': ['Alice', 'Bob', 'Carol', 'David', 'Eve', 'Frank', 'Gina', 'Hank', 'Ivy', 'Jack', 'Kathy', 'Larry', 'Mona', 'Nick', 'Oscar', 'Pam', 'Quincy', 'Rita', 'Sam', 'Tina'],
        'Age': [25, -5, 30, 128, 29, 20, 42, 33, 37, 120, 44, 15, -10, 66, 31, 40, 25, 50, 60, 35],
        'Gender': ['F', 'M', 'F', 'M', 'F', 'M', 'F', 'M', 'F', 'M', 'F', 'M', 'F', 'M', 'M', 'F', 'M', 'F', 'M', 'F'],
        'Income': [50000.0, 55000.0, float('nan'), 60000.0, 65000.0, float('nan'), 71000.0, -10000.0, float('nan'), 80000.0, 85000.0, 90000.0, 95000.0, 100000.0, 10000.0, -20000.0, 45000.0, float('nan'), 120000.0, 40000.0]}
df = pd.DataFrame(dat

'The cleaned data is stored in the df_cleaned DataFrame.'

In [36]:
df = pd.DataFrame(dummy_data)
df_cleaned = df.dropna().reset_index(drop=True)
df_cleaned['Age'] = df_cleaned['Age'].apply(lambda x: abs(x))
df_cleaned['Income'] = df_cleaned['Income'].apply(lambda x: abs(x))
df_cleaned

,Name,Age,Gender,Income
0,Alice,25,F,50000.0
1,Bob,5,M,55000.0
2,David,128,M,60000.0
3,Eve,29,F,65000.0
4,Gina,42,F,71000.0
5,Hank,33,M,10000.0
6,Jack,120,M,80000.0
7,Kathy,44,F,85000.0
8,Larry,15,M,90000.0
9,Mona,10,F,95000.0


In [31]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 



> Entering new  chain...
I can use the `sorted()` function to sort the list of customers. I will need to provide a key function that specifies the sorting order based on last name and then first name.
Action: Python_REPL
Action Input: sorted([['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']], key=lambda x: (x[1], x[0]))
Observation: 
Thought:The customers have been sorted by last name and then first name.
Final Answer: [['Jen', 'Ayai'], ['Harrison', 'Chase'], ['Lang', 'Chain'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Dolly', 'Too']]

> Finished chain.


"[['Jen', 'Ayai'], ['Harrison', 'Chase'], ['Lang', 'Chain'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Dolly', 'Too']]"

In [32]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nI

In [9]:
conversation.predict(input="Hi, my name is Andrew.  i have a dutch-mennonite background.  settled as farmer in southern mn in late 1800s")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my last name
AI: I'm sorry, but I don't have access to personal information like your last name.
Human: Hi, my name is Andrew.  i have a dutch-mennonite background.  settled as farmer in southern mn in late 1800s
AI:

> Finished chain.


"That's fascinating, Andrew! It's great to learn about your Dutch-Mennonite background and your family's history as farmers in southern Minnesota in the late 1800s. Do you have any specific questions or topics you'd like to discuss?"

In [7]:
memory.buffer

"Human: Hi, my name is Andrew\nAI: Hello Andrew! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1+1 is equal to 2."

In [13]:
conversation.predict(input="Are there any dutch mennonites that settled in canada around that same time?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my last name
AI: I'm sorry, but I don't have access to personal information like your last name.
Human: Hi, my name is Andrew.  i have a dutch-mennonite background.  settled as farmer in southern mn in late 1800s
AI: That's fascinating, Andrew! It's great to learn about your Dutch-Mennonite background and your family's history as farmers in southern Minnesota in the late 1800s. Do you have any specific questions or topics you'd like to discuss?
Human: What do you think my last name is?
AI: I'm sorry, but I don't have access to perso

'Yes, there were Dutch Mennonites who settled in Canada around the same time. In the late 1800s, many Mennonites from various European countries, including the Netherlands, immigrated to Canada. They settled in different provinces, such as Manitoba, Saskatchewan, and Ontario, and established Mennonite communities there. These communities played a significant role in the development of agriculture and other industries in Canada.'

In [14]:
conversation.predict(input="how good were the dutch mennonites at farming compared to the rest of the world in the late 1800s?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my last name
AI: I'm sorry, but I don't have access to personal information like your last name.
Human: Hi, my name is Andrew.  i have a dutch-mennonite background.  settled as farmer in southern mn in late 1800s
AI: That's fascinating, Andrew! It's great to learn about your Dutch-Mennonite background and your family's history as farmers in southern Minnesota in the late 1800s. Do you have any specific questions or topics you'd like to discuss?
Human: What do you think my last name is?
AI: I'm sorry, but I don't have access to perso

"In the late 1800s, Dutch Mennonites were known for their expertise in farming. They had a strong agricultural tradition and were skilled in various farming techniques and practices. The Netherlands, as a country, has a long history of agricultural innovation and productivity. Dutch Mennonites, with their farming background, likely brought their knowledge and skills to their new settlements in places like southern Minnesota and Canada. However, it's important to note that farming practices and productivity can vary across regions and countries, so it's difficult to make a direct comparison to the rest of the world."

In [15]:
conversation.predict(input="did they help out developing agriculture in ukraine?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my last name
AI: I'm sorry, but I don't have access to personal information like your last name.
Human: Hi, my name is Andrew.  i have a dutch-mennonite background.  settled as farmer in southern mn in late 1800s
AI: That's fascinating, Andrew! It's great to learn about your Dutch-Mennonite background and your family's history as farmers in southern Minnesota in the late 1800s. Do you have any specific questions or topics you'd like to discuss?
Human: What do you think my last name is?
AI: I'm sorry, but I don't have access to perso

'Yes, Dutch Mennonites played a significant role in developing agriculture in Ukraine. In the late 18th and early 19th centuries, many Mennonites from the Netherlands migrated to Ukraine, particularly to the region known as the Chortitza Colony. They brought with them advanced agricultural techniques and knowledge, which greatly contributed to the development of agriculture in the area. The Mennonites in Ukraine were known for their successful farming practices, including the introduction of new crops, irrigation systems, and modern farming equipment. They played a crucial role in transforming the Ukrainian landscape and establishing prosperous agricultural communities.'

In [16]:
conversation.predict(input="how about in poland?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my last name
AI: I'm sorry, but I don't have access to personal information like your last name.
Human: Hi, my name is Andrew.  i have a dutch-mennonite background.  settled as farmer in southern mn in late 1800s
AI: That's fascinating, Andrew! It's great to learn about your Dutch-Mennonite background and your family's history as farmers in southern Minnesota in the late 1800s. Do you have any specific questions or topics you'd like to discuss?
Human: What do you think my last name is?
AI: I'm sorry, but I don't have access to perso

'Dutch Mennonites also played a role in developing agriculture in Poland. In the late 16th and early 17th centuries, some Mennonites from the Netherlands migrated to Poland, particularly to the region of West Prussia. They brought with them their agricultural expertise and contributed to the development of farming practices in the area. The Mennonites in Poland were known for their successful agricultural settlements and their ability to cultivate the land effectively. They introduced new farming techniques, improved irrigation systems, and helped establish prosperous agricultural communities in Poland.'

In [17]:
conversation.predict(input="where they better at farming the other immigrant eurpean groups that migrated to the US?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my last name
AI: I'm sorry, but I don't have access to personal information like your last name.
Human: Hi, my name is Andrew.  i have a dutch-mennonite background.  settled as farmer in southern mn in late 1800s
AI: That's fascinating, Andrew! It's great to learn about your Dutch-Mennonite background and your family's history as farmers in southern Minnesota in the late 1800s. Do you have any specific questions or topics you'd like to discuss?
Human: What do you think my last name is?
AI: I'm sorry, but I don't have access to perso

"It's difficult to make a direct comparison between Dutch Mennonites and other immigrant European groups in terms of farming skills and practices. Each group brought their own unique knowledge and experiences to the United States, and their success in farming would depend on various factors such as the specific region they settled in, the resources available to them, and their ability to adapt to the new environment. However, Dutch Mennonites were known for their strong agricultural tradition and expertise in farming, so they likely had a significant impact on the development of agriculture in the areas where they settled."

In [18]:
conversation.predict(input="What did some of their chidlren end up doing that did not go into farming?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my last name
AI: I'm sorry, but I don't have access to personal information like your last name.
Human: Hi, my name is Andrew.  i have a dutch-mennonite background.  settled as farmer in southern mn in late 1800s
AI: That's fascinating, Andrew! It's great to learn about your Dutch-Mennonite background and your family's history as farmers in southern Minnesota in the late 1800s. Do you have any specific questions or topics you'd like to discuss?
Human: What do you think my last name is?
AI: I'm sorry, but I don't have access to perso

"Some children of Dutch Mennonite families who did not go into farming pursued various other professions and careers. Some may have become teachers, doctors, lawyers, merchants, or artisans. Others may have been involved in trades such as carpentry, blacksmithing, or weaving. The specific occupations would have depended on the individual's interests, skills, and opportunities available in their respective communities."

In [19]:
conversation.predict(input="did any get into natural bodybuilding?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my last name
AI: I'm sorry, but I don't have access to personal information like your last name.
Human: Hi, my name is Andrew.  i have a dutch-mennonite background.  settled as farmer in southern mn in late 1800s
AI: That's fascinating, Andrew! It's great to learn about your Dutch-Mennonite background and your family's history as farmers in southern Minnesota in the late 1800s. Do you have any specific questions or topics you'd like to discuss?
Human: What do you think my last name is?
AI: I'm sorry, but I don't have access to perso

"It's difficult to say for certain whether any children of Dutch Mennonite families in the late 1800s got into natural bodybuilding, as specific information about individuals and their hobbies or interests is not available. However, it's possible that some individuals from Dutch Mennonite backgrounds may have pursued bodybuilding or fitness-related activities as a personal interest or hobby."

In [20]:
conversation.predict(input="are any data scientists today?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my last name
AI: I'm sorry, but I don't have access to personal information like your last name.
Human: Hi, my name is Andrew.  i have a dutch-mennonite background.  settled as farmer in southern mn in late 1800s
AI: That's fascinating, Andrew! It's great to learn about your Dutch-Mennonite background and your family's history as farmers in southern Minnesota in the late 1800s. Do you have any specific questions or topics you'd like to discuss?
Human: What do you think my last name is?
AI: I'm sorry, but I don't have access to perso

"It's possible that some individuals from Dutch Mennonite backgrounds are data scientists today. Data science is a growing field that requires a combination of analytical skills, programming knowledge, and domain expertise. People from various backgrounds and professions can transition into data science by acquiring the necessary skills and education. However, without specific information about individuals, it's difficult to determine if any Dutch Mennonites specifically are data scientists."